<a href="https://colab.research.google.com/github/nicolas-errandonea/TFG/blob/TFGcodes/RF_la_liga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#primero importamos la primera base de datos
from google.colab import drive
from matplotlib import pyplot as plt
import numpy as np 
import pandas as pd
import random
drive.mount('/content/gdrive', force_remount=True)
path = "/content/gdrive/My Drive/liga/liga_data.csv"
df = pd.read_csv(path)

Mounted at /content/gdrive


In [ ]:
from tensorflow import keras
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import numpy
from sklearn import metrics
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

In [ ]:
df2=df.copy()
cols_to_drop = ['Date','HomeTeam','AwayTeam','FTR', 'HR','AR','HR_A','AR_A']


df2.drop( columns = cols_to_drop, inplace = True)

In [ ]:
prediction_cols=['FTR']

x_train = df2[1140:]        
y_train = df[1140:][prediction_cols]
x_test = df2[0:1140]       
y_test = df[0:1140][prediction_cols]

In [ ]:
clf=RandomForestClassifier(n_estimators=2000,warm_start=False,ccp_alpha=0.001)

In [ ]:
clf.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(ccp_alpha=0.001, n_estimators=2000)

In [ ]:
y_pred=clf.predict(x_test)
print(y_pred)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

[3 1 1 ... 1 3 3]
Accuracy: 0.5105263157894737


In [ ]:
result=(y_test.to_numpy())
print(type(result))

<class 'numpy.ndarray'>


In [ ]:
#comprobación de mi codigo
hits=0
total=0
for i in range(len(y_pred)):
  if((y_pred[i]==[0,0,0]).all()):
    i
  else:
    total=total+1
    if(((y_pred[i]-result[i])==[0,0,0]).all()):
      hits=hits+1
print(hits/total)

0.5105263157894737


In [ ]:
#acierto de las casas de apuestas
total=len(y_test.index)
aciertos=0
for i in y_test.index:
   minimo=min(df2.B365H[i],df2.B365D[i],df2.B365A[i])
   if(df.FTR[i]==1 and minimo==df2.B365H[i]):
     aciertos=aciertos+1
   if(df.FTR[i]==2 and minimo==df2.B365D[i]):
     aciertos=aciertos+1
     
   if(df.FTR[i]==3 and minimo==df2.B365A[i]):
     aciertos=aciertos+1
print(aciertos/total)

0.5236842105263158


In [ ]:
feature_imp = pd.Series(clf.feature_importances_,index=list(df2)).sort_values(ascending=False)
print((feature_imp))

B365H            0.072064
B365A            0.068543
B365D            0.035847
AS_A             0.033564
AST_A            0.033176
HST              0.030247
HS               0.030089
HC_A             0.029895
AF_A             0.029769
HC               0.029540
AC               0.029507
HF_A             0.029150
HF               0.028970
AF               0.028092
HS_A             0.027857
AS               0.027561
AY               0.026370
HST_A            0.026113
AC_A             0.025829
FTHG             0.025307
home_points      0.025185
AST              0.024828
HY               0.024506
HY_A             0.023981
away_points_A    0.023664
AY_A             0.023353
FTAG_A           0.023343
home_points_A    0.022457
away_points      0.022394
FTHG_A           0.022257
FTAG             0.020905
HTHG             0.020452
HTHG_A           0.019607
HTAG_A           0.017899
HTAG             0.017677
dtype: float64


# **Searching for the best random forest**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 4000, num = 8)]
# Number of features to consider at every split
max_features = ['auto']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 170, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5,7, 10,12]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 4,6,7,10,12]
# Method of selecting samples for training each tree
bootstrap = [True]
warm_start=[True]
ccp_alpha=[0.003,0.007,0.005,0.0025]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'warm_start':warm_start,
               'ccp_alpha':ccp_alpha}
print(random_grid)

{'n_estimators': [200, 742, 1285, 1828, 2371, 2914, 3457, 4000], 'max_features': ['auto'], 'max_depth': [10, 26, 42, 58, 74, 90, 106, 122, 138, 154, 170, None], 'min_samples_split': [2, 5, 7, 10, 12], 'min_samples_leaf': [2, 4, 6, 7, 10, 12], 'bootstrap': [True], 'warm_start': [True], 'ccp_alpha': [0.003, 0.007, 0.005, 0.0025]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 250, cv = 4, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(x_train, y_train)

Fitting 4 folds for each of 250 candidates, totalling 1000 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=4, estimator=RandomForestClassifier(), n_iter=250,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True],
                                        'ccp_alpha': [0.003, 0.007, 0.005,
                                                      0.0025],
                                        'max_depth': [10, 26, 42, 58, 74, 90,
                                                      106, 122, 138, 154, 170,
                                                      None],
                                        'max_features': ['auto'],
                                        'min_samples_leaf': [2, 4, 6, 7, 10,
                                                             12],
                                        'min_samples_split': [2, 5, 7, 10, 12],
                                        'n_estimators': [200, 742, 1285, 1828,
                                                         2371, 2914, 3457,
                                     

In [ ]:
rf_random.best_params_

{'bootstrap': True,
 'ccp_alpha': 0.0025,
 'max_depth': 42,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 200,
 'warm_start': True}

In [ ]:
def evaluate(model, x_test, y_test):
  result=(y_test.to_numpy())
  y_pred = model.predict(x_test)
  hits=0
  total=0
  for i in range(len(y_pred)):
    total=total+1
    if((y_pred[i]-result[i])==[0]):
      hits=hits+1
  print(hits/total)
  accuracy=hits/total
  return accuracy

In [ ]:
base_model = RandomForestClassifier(n_estimators = 1000,warm_start=False,ccp_alpha=0.005)
base_model.fit(x_train, y_train)
base_accuracy = evaluate(base_model, x_test, y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.5166666666666667


In [ ]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, x_test, y_test)

0.5175438596491229


In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 400, num = 4)]
# Number of features to consider at every split
max_features = ['auto']
# Maximum number of levels in tree
max_depth = [40,50,60]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2,4]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2,4]
# Method of selecting samples for training each tree
bootstrap = [True]
warm_start=[True]
ccp_alpha=[0.0025,0.005]
max_samples=[None]
min_weight_fraction_leaf=[0.05,0.5,0]
max_leaf_nodes=[None,100,1000,]
# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'warm_start':warm_start,
               'ccp_alpha':ccp_alpha,
               'max_samples':max_samples,
               'min_weight_fraction_leaf':min_weight_fraction_leaf,
               'max_leaf_nodes':max_leaf_nodes}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
grid_search.fit(x_train, y_train)
grid_search.best_params_

Fitting 3 folds for each of 1152 candidates, totalling 3456 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'bootstrap': True,
 'ccp_alpha': 0.0025,
 'max_depth': 60,
 'max_features': 'auto',
 'max_leaf_nodes': 100,
 'max_samples': None,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0,
 'n_estimators': 400,
 'warm_start': True}

In [ ]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, x_test, y_test)

0.5105263157894737


In [ ]:
base_modell = RandomForestClassifier(n_estimators = 400,max_leaf_nodes=100,bootstrap=True ,max_depth=60,max_samples=None,max_features='auto',min_samples_leaf=2,min_samples_split=2,warm_start=True,ccp_alpha=0.0025,min_weight_fraction_leaf=0)
base_modell.fit(x_train, y_train)
base_accuracy = evaluate(base_modell, x_test, y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.5175438596491229
